<a href="https://colab.research.google.com/github/RobAltena/AdventOfCode2025/blob/main/Day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import Counter
import requests

data = """.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
"""

data = requests.get('https://raw.githubusercontent.com/RobAltena/AdventofCode2025/refs/heads/main/advent_day7_input.txt').text


arr = np.array([list(line) for line in data.split()])
np.set_printoptions(linewidth=200, threshold=np.inf)

In [ ]:
s_index = np.where(arr[0] == 'S')[0][0]

beams = set([s_index])
row = 1
splits = 0

while row < arr.shape[0]:
  new_beams = set()
  for beam in beams:
    match arr[row, beam]:
      case '.':
        new_beams.add(beam)
      case '^':
        new_beams.update([beam-1, beam+1])
        splits += 1
  row += 1
  beams = new_beams

print(f"Day 7, part 1: {splits}")

In [ ]:
beams = Counter({s_index : 1})
row = 1


while row < arr.shape[0]:
  new_beams = Counter()

  for beam, count in beams.items():
    match arr[row, beam]:
      case '.':
        new_beams.update({beam : count})
      case '^':
        new_beams.update({beam-1 : count, beam+1 : count})

  row += 1
  beams = new_beams

print(f"Day 7, part 2: {beams.total()}")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import numpy as np
from collections import Counter
from matplotlib.colors import LinearSegmentedColormap, Normalize, LogNorm # Import LogNorm

# --- Configuration for animation speed (for display, not data generation here) ---
FRAME_SKIP = 1 # Animate every Nth row. Set to 1 for every row.

# --- Initialization for beam simulation and tracking max count ---
s_index = np.where(arr[0] == 'S')[0][0]

# This list will store all the processed grid data for the animation frames.
# Each frame will be a numpy array representing the cumulative beam counts.
all_frames_data = []
max_count_at_any_single_pos = 1 # Initialize to 1 to avoid log(0) issues, as min count will be 1

beams = Counter({s_index : 1})

# Initialize a cumulative grid to build the 'Christmas tree' effect
cumulative_grid = np.zeros_like(arr, dtype=float)

# --- Main simulation loop to generate frame data ---
for r_idx in range(arr.shape[0]):
    # Update max_count_at_any_single_pos for the current state of beams
    if beams:
        current_row_max = max(beams.values())
        if current_row_max > max_count_at_any_single_pos:
            max_count_at_any_single_pos = current_row_max

    # Create a grid for the current row's display values (background + beams)
    current_display_grid = np.zeros_like(arr, dtype=float)
    # First, mark splitters if they are not overridden by a beam in this row.
    for c_idx in range(arr.shape[1]):
        if arr[r_idx, c_idx] == '^':
            current_display_grid[r_idx, c_idx] = 0.1 # Mark splitter with a small value (distinct from 0)

    # Mark active beam positions for the current row with their actual counts
    for beam_pos, count in beams.items():
        if 0 <= r_idx < arr.shape[0] and 0 <= beam_pos < arr.shape[1]:
            # Beams override splitter value if they are on a splitter
            current_display_grid[r_idx, beam_pos] = float(count)

    # Update the cumulative grid (keeping track of the maximum value encountered for each cell)
    for c_idx in range(arr.shape[1]):
        # Keep the maximum value seen in this cell across all rows so far
        cumulative_grid[r_idx, c_idx] = max(cumulative_grid[r_idx, c_idx], current_display_grid[r_idx, c_idx])

    # Append a copy of the cumulative grid if it's a frame to be skipped or the first/last row
    if r_idx % FRAME_SKIP == 0 or r_idx == 0 or r_idx == arr.shape[0] - 1:
        # Store a copy of the current cumulative state and metadata
        all_frames_data.append((cumulative_grid.copy(), r_idx, beams.total()))

    # Calculate new_beams for the next row, if not the last row
    if r_idx < arr.shape[0] - 1:
        new_beams = Counter()
        for beam_pos, count in beams.items():
            if 0 <= beam_pos < arr.shape[1]: # Ensure beam_pos is within bounds
                if arr[r_idx + 1, beam_pos] == '.':
                    new_beams.update({beam_pos : count})
                elif arr[r_idx + 1, beam_pos] == '^':
                    if (beam_pos - 1 >= 0):
                        new_beams.update({beam_pos - 1 : count})
                    if (beam_pos + 1 < arr.shape[1]):
                        new_beams.update({beam_pos + 1 : count})
        beams = new_beams.copy()


# --- Create the animation ---
fig_width = max(arr.shape[1] / 10, 5)
fig_height = max(arr.shape[0] / 10, 5)

fig, ax = plt.subplots(figsize=(fig_width, fig_height))

# Create LogNorm for color scaling. min_val for log needs to be > 0.
# Max value is max_count_at_any_single_pos.
# We map 0.0 to the lowest color, 0.1 (splitters) to a slightly higher color.
# Actual counts will then scale logarithmically.
# It's important to set vmin to a very small positive number if LogNorm is used,
# as log(0) is undefined. Values of 0 in the grid will be treated as below vmin.
norm = LogNorm(vmin=0.01, vmax=max_count_at_any_single_pos) # Using 0.01 to distinguish from 0

im = ax.imshow(all_frames_data[0][0], cmap='coolwarm', animated=True, norm=norm) # Apply normalization
ax.set_xticks([])
ax.set_yticks([])

title_text = ax.set_title(f"Day 7, Part 2 - Row: {all_frames_data[0][1]}, Beams: {all_frames_data[0][2]}")

def update(frame_data):
    grid_data, current_row, total_beams = frame_data
    im.set_array(grid_data)
    title_text.set_text(f"Day 7, Part 2 - Row: {current_row}, Beams: {total_beams}")
    return im, title_text

ani = FuncAnimation(fig, update, frames=all_frames_data, blit=True, interval=100)

# Save the animation as a GIF
# You might need to install 'imagemagick' on your system for this to work.
# On Colab, you can run: !apt-get update && apt-get install imagemagick
ani.save('beam_animation.gif', writer='imagemagick', fps=10)

HTML(ani.to_jshtml())